# **Extracting Information from Legal Documents Using RAG**

## **Objective**

This project has been worked on and completed by Manish Parashar, Vaishnavi Rachita and Karthik Ramesh

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [11]:
## The following libraries might be useful
# !pip install -q langchain-openai
# !pip install -U -q langchain-community
# !pip install -U -q langchain-chroma
# !pip install -U -q datasets
# !pip install -U -q ragas
# !pip install -U -q rouge_score

In [42]:
# Import essential libraries


import os, json, glob, re, zipfile, logging, openai, shutil
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from datasets import load_dataset


from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

from ragas import evaluate
from datasets import Dataset
from ragas import evaluate as ragas_evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision, context_recall
import nltk
try:
    _ = nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\debas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [18]:
# Load the files as documents

zip_path = "rag_legal.zip"

# Extracting in current directory (same as zip)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    files = zip_ref.namelist()
    for file in tqdm(files, desc="Extracting", unit="file"):
        zip_ref.extract(file, ".")   # "." means current dir

print("✅ Extraction complete!")

extract_path = 'rag_Legal/corpus'

all_documents = []

# Iterating over all subfolders & .txt files
for folder in ["contractnli", "cuad", "maud", "privacy_qa"]:
    folder_path = os.path.join(extract_path, folder)
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    
    for file_path in txt_files:
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            docs = loader.load()
            # Adding metadata
            for d in docs:
                d.metadata["source"] = file_path
                d.metadata["category"] = folder
            all_documents.extend(docs)
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

print(f"Loaded {len(all_documents)} documents")

Extracting: 100%|█████████████████████████████████████████████████████████████████| 702/702 [00:02<00:00, 236.44file/s]


✅ Extraction complete!
Loaded 698 documents


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [19]:
# Clean and preprocess the data

logging.basicConfig(filename="preprocess.log", level=logging.INFO,
                    format="%(asctime)s | %(levelname)s | %(message)s")

# Regex patterns
EMAIL = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', re.I)
PHONE = re.compile(r'\b(?:\+?\d{1,3}[\s-]?)?(?:\(?\d{2,4}\)?[\s-]?)?\d{3,4}[\s-]?\d{4}\b')
URL   = re.compile(r'\b(?:https?://|www\.)\S+\b', re.I)
SPACE = re.compile(r'\s+')

# Adjusting stopwords: remove common ones, but keep legal terms
LEGAL_TERMS = {"shall", "agreement", "party", "contract", "confidentiality"}
STOPWORDS = set(ENGLISH_STOP_WORDS) - LEGAL_TERMS

def clean_text(s, keep_legal_symbols=True):
    if not isinstance(s, str) or not s.strip():
        return ""
    s = EMAIL.sub(" ", s); s = PHONE.sub(" ", s); s = URL.sub(" ", s)  # remove noise
    s = s.lower()  # normalize case
    if keep_legal_symbols:
        s = re.sub(r"[^a-z0-9\s\,\.\-\/\$\%\:\;\?]", " ", s)  # keep useful legal symbols
    else:
        s = re.sub(r"[^a-z0-9\s]", " ", s)  # or else remove all symbols, keep only letters, digits, and spaces.

    # Normalize possessives (company's -> company)
    s = re.sub(r"\b([a-z]+)'s\b", r"\1", s)
    
    # Remove Roman numerals (i, ii, iii, iv, etc.)
    s = re.sub(r"\b([ivx]{1,4})\b", " ", s)
    
    tokens = [t for t in s.split() if t not in STOPWORDS]  #split the text into words, remove stop words except for the legal terms
    return SPACE.sub(" ", " ".join(tokens)).strip()

def preprocess_documents(docs, keep_legal_symbols=True):
    cleaned, errors = [], []
    # cleaning each document
    for d in tqdm(docs, desc="cleaning documents"):  
        try:
            txt = clean_text(d.page_content, keep_legal_symbols)
            if not txt:
                raise ValueError("empty after cleaning")  #raise an error if the documents become empty after cleaning
            # update the metadata of the files
            md = dict(d.metadata or {})
            md.update({
                "cleaned": True,
                "cleaned_at": datetime.utcnow().isoformat(timespec="seconds"),
                "orig_length": len(d.page_content.split()),
                "clean_length": len(txt.split())
            })
            cleaned.append(Document(page_content=txt, metadata=md))
        # if cleaning fails, log the error    
        except Exception as e:
            errors.append({
                "source": (d.metadata or {}).get("source"),
                "category": (d.metadata or {}).get("category"),
                "error": str(e)
            })
            logging.error(f"Skipping doc: {e}")
    return cleaned, pd.DataFrame(errors)

# ---- Run preprocessing ----
cleaned_documents, error_df = preprocess_documents(all_documents)

print(f"Cleaned: {len(cleaned_documents)}")
print(f"Skipped: {0 if error_df.empty else len(error_df)}")

cleaning documents:   0%|                                                                      | 0/698 [00:00<?, ?it/s]C:\Users\debas\AppData\Local\Temp\ipykernel_26040\3136069213.py:47: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "cleaned_at": datetime.utcnow().isoformat(timespec="seconds"),
cleaning documents: 100%|████████████████████████████████████████████████████████████| 698/698 [00:22<00:00, 30.68it/s]

Cleaned: 698
Skipped: 0


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [20]:
# Calculate the average, maximum and minimum document length.

doc_lengths = [len(d.page_content.split()) for d in cleaned_documents]

avg_len = sum(doc_lengths)/len(doc_lengths)
max_len = max(doc_lengths)
min_len = min(doc_lengths)

print("Document Length Statistics (in words):")
print(f"Average length: {avg_len:.2f}")
print(f"Maximum length: {max_len}")
print(f"Minimum length: {min_len}")


Document Length Statistics (in words):
Average length: 8803.87
Maximum length: 83269
Minimum length: 139


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [21]:
# Find frequency of occurence of words

from collections import Counter


# tokens we want to skip in analysis
PUNCT_TOKENS = {",", ".", "-", "/", ":", ";", "?", "!"}
ROMAN_RE = re.compile(r"^[ivxlcdm]{1,6}$")  # matches i, ii, iii, iv, etc.

def is_valid_token(tok: str) -> bool:
    # skipping punctuation-only tokens
    if tok in PUNCT_TOKENS:
        return False
    # skipping lone letters (like 's', 'b') but keep symbols like $, %
    if len(tok) == 1 and tok not in {"$", "%"}:
        return False
    # skipping roman numerals (common in section headings)
    if ROMAN_RE.fullmatch(tok):
        return False
    return True


# Flattening all tokens
all_tokens = [t for d in cleaned_documents for t in d.page_content.split()]

# Applying filter
word_counts = Counter(t for t in all_tokens if is_valid_token(t))# Flatten all tokens

# 20 most common words
most_common_20 = word_counts.most_common(20)

# 20 least common words (words with lowest frequency)
least_common_20 = sorted(word_counts.items(), key=lambda x: x[1])[:20]

print("Top 20 Most Common Words:")
for word, count in most_common_20:
    print(f"{word}: {count}")

print("\nBottom 20 Least Common Words:")
for word, count in least_common_20:
    print(f"{word}: {count}")

Top 20 Most Common Words:
company: 140837
shall: 104502
section: 74873
agreement: 67836
parent: 52087
party: 46084
material: 33439
date: 31092
merger: 29329
respect: 28355
applicable: 26680
including: 26300
subsidiaries: 25202
time: 23928
prior: 23316
stock: 22629
agreement,: 21878
information: 21359
effective: 21039
pursuant: 20907

Bottom 20 Least Common Words:
matter:: 1
exemptions.: 1
secret:: 1
embodying,: 1
inurement.: 1
center/centre: 1
monnaie: 1
disclosee,: 1
anybody: 1
candidacy: 1
grateful: 1
sohet: 1
koen: 1
gerven: 1
sa/nv: 1
cvba: 1
peratieve: 1
vennootschap: 1
beperkte: 1
aansprakelijkheid/soci: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [22]:
# Transform the page contents of documents

# Selecting the first 10 (or fewer if corpus is small)
first10_idx = list(range(min(10, len(cleaned_documents))))

# Building texts and human-readable labels (use file name if available)
texts_10 = []
labels_10 = []

for i in first10_idx:
    # adding the document content
    texts_10.append(cleaned_documents[i].page_content)
    
    # adding the document label (use file name if available)
    source = (cleaned_documents[i].metadata or {}).get("source", f"doc_{i}")
    labels_10.append(os.path.basename(source))

# TF-IDF: including unigrams + bigrams; keeping lowercase=False since we already normalized
tfidf_10 = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)  #min_df =2 to prevent words that appear in only one document
X_10 = tfidf_10.fit_transform(texts_10)   # shape: [n_docs x vocab]

# Compute similarity scores


# Cosine similarity in [0, 1]; 1 = identical TF-IDF vectors
sim_10 = cosine_similarity(X_10, dense_output=True)

# Put into a labeled DataFrame for readability
sim_df_10 = pd.DataFrame(sim_10, index=labels_10, columns=labels_10)

print("Cosine similarity (first 10 docs):")
display(sim_df_10.round(3))

Cosine similarity (first 10 docs):


01_Bosch-Automotive-Service-Solutions-Mutual-Non-Disclosure-Agreement-7-12-17.txt  \
01_Bosch-Automotive-Service-Solutions-Mutual-No...                                              1.000                                   
12032018_NDA_The_20Munt_EN.txt                                                                  0.478                                   
5-NSK-Confidentiality-Agreement-for-Suppliers.txt                                               0.610                                   
ADVANIDE-NON-DISCLOSURE-AGREEMENT.txt                                                           0.517                                   
AfriGIS_Client-NDA_Template_2019.txt                                                            0.554                                   
AGProjects-NDA.txt                                                                              0.607                                   
amc-general-mutual-non-disclosure-agreement-en-...                                              0.607                                   
Aspiegel_NDA_template.txt                                                                       0.730                                   
BCG-Mutual-NDA.txt                                                                              0.551                                   
BT_NDA.txt                                                                                      0.621                                   

                                                    12032018_NDA_The_20Munt_EN.txt  \
01_Bosch-Automotive-Service-Solutions-Mutual-No...                           0.478   
12032018_NDA_The_20Munt_EN.txt                                               1.000   
5-NSK-Confidentiality-Agreement-for-Suppliers.txt                            0.470   
ADVANIDE-NON-DISCLOSURE-AGREEMENT.txt                                        0.352   
AfriGIS_Client-NDA_Template_2019.txt                                         0.409   
AGProjects-NDA.txt                                                           0.429   
amc-general-mutual-non-disclosure-agreement-en-...                           0.387   
Aspiegel_NDA_template.txt                                                    0.456   
BCG-Mutual-NDA.txt                                                           0.535   
BT_NDA.txt                                                                   0.491   

                                                    5-NSK-Confidentiality-Agreement-for-Suppliers.txt  \
01_Bosch-Automotive-Service-Solutions-Mutual-No...                                              0.610   
12032018_NDA_The_20Munt_EN.txt                                                                  0.470   
5-NSK-Confidentiality-Agreement-for-Suppliers.txt                                               1.000   
ADVANIDE-NON-DISCLOSURE-AGREEMENT.txt                                                           0.691   
AfriGIS_Client-NDA_Template_2019.txt                                                            0.743   
AGProjects-NDA.txt                                                                              0.750   
amc-general-mutual-non-disclosure-agreement-en-...                                              0.744   
Aspiegel_NDA_template.txt                                                                       0.756   
BCG-Mutual-NDA.txt                                                                              0.503   
BT_NDA.txt                                                                                      0.767   

                                                    ADVANIDE-NON-DISCLOSURE-AGREEMENT.txt  \
01_Bosch-Automotive-Service-Solutions-Mutual-No...                                  0.517   
12032018_NDA_The_20Munt_EN.txt                                                      0.352   
5-NSK-Confidentiality-Agreement-for-Suppliers.txt                                   0.691   
ADVANIDE-NON-DISCLOSURE-AGREEMENT.txt                                

In [23]:
# create a list of 10 random integers

rng = np.random.default_rng(42)   # seed for reproducibility
n_docs = len(cleaned_documents)
k = min(10, n_docs)               # 10 or less dependening on the document availability

rand10_idx = rng.choice(n_docs, size=k, replace=False).tolist()
print("Random 10 indices:", rand10_idx)

# Building texts and labels using a for-loop
texts_r = []
labels_r = []

for i in rand10_idx:
    # text content
    texts_r.append(cleaned_documents[i].page_content)
    # human-readable label (file name if available)
    src = (cleaned_documents[i].metadata or {}).get("source", f"doc_{i}")
    labels_r.append(os.path.basename(src))

Random 10 indices: [59, 534, 61, 452, 303, 300, 485, 65, 140, 595]


In [24]:
# Compute similarity scores for 10 random documents

# Vectorize (unigram + bigram helps catch legal phrases; lowercase already done in cleaning)
tfidf_r = TfidfVectorizer(ngram_range=(1, 2), min_df=2, lowercase=False)
X_r = tfidf_r.fit_transform(texts_r)   # shape: [k x vocab]

# Cosine similarity matrix in [0,1]
sim_r = cosine_similarity(X_r, dense_output=True)

# labeled DataFrame for readability
sim_df_r = pd.DataFrame(sim_r, index=labels_r, columns=labels_r)

print("Cosine similarity (random 10 docs):")
display(sim_df_r.round(3))

Cosine similarity (random 10 docs):


NDA-SqillerBetaTesters.txt  \
NDA-SqillerBetaTesters.txt                                               1.000   
WatchitMediaInc_20061201_8-K_EX-10.1_4148672_EX...                       0.083   
NDA_-_GWI_-_Technohubs_version_V1.txt                                    0.645   
SEASPINEHOLDINGSCORP_10_10_2018-EX-10.1-SUPPLY ...                       0.412   
ImpresseCorp_20000322_S-1A_EX-10.11_5199234_EX-...                       0.428   
IMMUNOMEDICSINC_08_07_2019-EX-10.1-PROMOTION AG...                       0.402   
TcPipelinesLp_20160226_10-K_EX-99.12_9454048_EX...                       0.098   
NDA_Street_Stream_Franchise.txt                                          0.508   
BABCOCK_WILCOXENTERPRISES,INC_08_04_2015-EX-10....                       0.249   
DSP_Group_Synaptics_Incorporated.txt                                     0.199   

                                                    WatchitMediaInc_20061201_8-K_EX-10.1_4148672_EX-10.1_Content License Agreement.txt  \
NDA-SqillerBetaTesters.txt                                                                      0.083                                    
WatchitMediaInc_20061201_8-K_EX-10.1_4148672_EX...                                              1.000                                    
NDA_-_GWI_-_Technohubs_version_V1.txt                                                           0.100                                    
SEASPINEHOLDINGSCORP_10_10_2018-EX-10.1-SUPPLY ...                                              0.106                                    
ImpresseCorp_20000322_S-1A_EX-10.11_5199234_EX-...                                              0.133                                    
IMMUNOMEDICSINC_08_07_2019-EX-10.1-PROMOTION AG...                                              0.131                                    
TcPipelinesLp_20160226_10-K_EX-99.12_9454048_EX...                                              0.111                                    
NDA_Street_Stream_Franchise.txt                                                                 0.139                                    
BABCOCK_WILCOXENTERPRISES,INC_08_04_2015-EX-10....                                              0.105                                    
DSP_Group_Synaptics_Incorporated.txt                                                            0.091                                    

                                                    NDA_-_GWI_-_Technohubs_version_V1.txt  \
NDA-SqillerBetaTesters.txt                                                          0.645   
WatchitMediaInc_20061201_8-K_EX-10.1_4148672_EX...                                  0.100   
NDA_-_GWI_-_Technohubs_version_V1.txt                                               1.000   
SEASPINEHOLDINGSCORP_10_10_2018-EX-10.1-SUPPLY ...                                  0.402   
ImpresseCorp_20000322_S-1A_EX-10.11_5199234_EX-...                                  0.443   
IMMUNOMEDICSINC_08_07_2019-EX-10.1-PROMOTION AG...                                  0.404   
TcPipelinesLp_20160226_10-K_EX-99.12_9454048_EX...                                  0.167   
NDA_Street_Stream_Franchise.txt                                                     0.520   
BABCOCK_WILCOXENTERPRISES,INC_08_04_2015-EX-10....                                  0.264   
DSP_Group_Synaptics_Incorporated.txt                                                0.226   

                                                    SEASPINEHOLDINGSCORP_10_10_2018-EX-10.1-SUPPLY AGREEMENT.txt  \
NDA-SqillerBetaTesters.txt                                                                      0.412              
WatchitMediaInc_20061201_8-K_EX-10.1_4148672_EX...                                              0.106              
NDA_-_GWI_-_Technohubs_version_V1.txt                                                           0.402              
SEASPINEHOLDINGSCORP_10_10_2018-EX-10.1-SUPPLY ...                                              1.000              
ImpresseCorp_20000322_S-1A

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [25]:
# Process files and generate chunks

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
    separators=["\n\n", "\n", ". ", " ", ""],  # fallback to character split
    length_function=len
)

# Spliting documents and adding helpful metadata
chunked_documents = []
for d in tqdm(cleaned_documents, desc="Chunking documents"):
    # spliting returns new Document objects with page_content set per chunk
    chunks = splitter.split_text(d.page_content)

    # attaching metadata to each chunk
    for idx, chunk_text in enumerate(chunks):
        md = dict(d.metadata or {})
        md.update({
            "chunk_index": idx,
            "chunk_id": f"{md.get('source', 'unknown')}::chunk_{idx}",  # unique id and index for unique retreival
        })
        chunked_documents.append(Document(page_content=chunk_text, metadata=md))

print(f"Chunks created: {len(chunked_documents)} "
      f"(from {len(cleaned_documents)} documents)")

Chunking documents: 100%|███████████████████████████████████████████████████████████| 698/698 [00:05<00:00, 124.99it/s]

Chunks created: 84930 (from 698 documents)


## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [26]:
# Fetch your OPENAI API Key as an environment variable

with open("OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

# Update the OpenAI API key by updating the environment variable
os.environ["OPENAI_API_KEY"] = openai.api_key

In [27]:
# Initialise an embedding function

embeddings_model = OpenAIEmbeddings()

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [28]:
# Add Chunks to vector DB

vector_db_dir = "vector_db_chroma"

# wipe any previous index
if os.path.exists(vector_db_dir):
    shutil.rmtree(vector_db_dir)
os.makedirs(vector_db_dir, exist_ok=True)

# build + persist
vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings_model,
    persist_directory=vector_db_dir
)
vectorstore.persist()

# count chunks (Chroma exposes its internal collection)
count = vectorstore._collection.count()
print(f"✅ Indexed {count} chunks into: {vector_db_dir}")

# reload later
vectorstore = Chroma(
    embedding_function=embeddings_model,
    persist_directory=vector_db_dir
)



✅ Indexed 84930 chunks into: vector_db_chroma


C:\Users\debas\AppData\Local\Temp\ipykernel_26040\3191944215.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
C:\Users\debas\AppData\Local\Temp\ipykernel_26040\3191944215.py:23: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [35]:
# Create a RAG chain

# Create a retriever (what & how many chunks to fetch)
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# Initialize the LLM (use any model you prefer)
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)  # deterministic for legal QA

prompt = ChatPromptTemplate.from_template(
    "You are a legal assistant. From the provided context, return ONLY the verbatim sentence(s) "
    "that directly answer the user's question.\n"
    "Do not add commentary, do not summarize, do not paraphrase.\n"
    "If no relevant sentence is found, respond with: No relevant sentence found.\n\n"
    "Question: {input}\n\nContext:\n{context}\n\n"
    "Relevant Sentences:"
)

# This chain knows how to turn List[Document] -> {context} string
combine_chain = create_stuff_documents_chain(llm, prompt)

# Retrieval chain = retriever -> combine_chain
rag_chain = create_retrieval_chain(retriever, combine_chain)

print("RAG chain ready")

RAG chain ready


#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [36]:
# Create a function for question answering

def extract_sentences(question, k=6):
    
    retriever.search_kwargs["k"] = k
    result = rag_chain.invoke({"input": question})

    sentences = (result.get("answer") or "").strip()
    print("\nRelevant Sentences:\n")
    print(sentences)

    print("\nRetrieved Chunks (transparency):")
    for i, doc in enumerate(result.get("context", []), 1):
        print(f"\nSource {i}:")
        print("file:", doc.metadata.get("source"))
        print(doc.page_content[:300], "...")
    
    return sentences

In [37]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"

question = (
    "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; "
    "Does the document indicate that the Agreement does not grant the Receiving Party any rights "
    "to the Confidential Information?"
)

print("\nAnswer:")
extract_sentences(question, k=6)


Answer:

Relevant Sentences:

No relevant sentence found.

Retrieved Chunks (transparency):

Source 1:
file: rag_Legal/corpus\cuad\ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT.txt
. 10.2 non-disclosure. receiving party agrees copy use disclosing party s confidential information purposes contemplated agreement, b maintain confidential, exercise reasonable precautions prevent unauthorized access, use disclosure, c disclose party receiving party s employees contractors need know ...

Source 2:
file: rag_Legal/corpus\contractnli\thoughtbot-mutual-nda.txt
. limiting foregoing, receiving party shall measures receiving party takes protect highly confidential information shall employees, any, access confidential information sign nonuse non-disclosure agreement content substantially similar provi-sions hereof, prior disclosure confidential information em ...

Source 3:
file: rag_Legal/corpus\maud\Xilinx, Inc._Advanced Micro Devices, Inc..txt
receives party effect party execute

'No relevant sentence found.'

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [40]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set

def load_all_benchmarks(benchmark_dir="rag_Legal/benchmarks"):  # <- folder where your 4 JSONs are stored
    # find all *.json files in the benchmark folder
    json_files = glob.glob(os.path.join(benchmark_dir, "*.json"))
    if not json_files:
        # if no files found, throw a clear error
        raise FileNotFoundError(
            f"No JSON files found in: {benchmark_dir}. "
            "Check the path (did you mean 'benchmarks' vs 'benchmark'?)."
        )

    # show which benchmark JSON files were detected
    print("Found benchmark files:")
    for p in json_files:
        print(" -", os.path.basename(p))

    # containers to collect outputs
    rows = []             # will hold one record per (question, answer, snippet)
    questions_all = []    # list of all questions (can include duplicates)
    answers_all = []      # list of all answers (can include duplicates)
    qa_dict = defaultdict(list)  # dictionary {question -> [unique answers]}

    # iterate over each JSON file
    for path in json_files:
        dataset = os.path.splitext(os.path.basename(path))[0]  # dataset name = filename w/o .json
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)  # load the JSON

        # each file has a list of test cases
        for t in data.get("tests", []):
            q = (t.get("query") or "").strip()  # the question text
            if q:
                questions_all.append(q)         # add question to global list

            # each test has one or more "snippets" (answers + metadata)
            for s in (t.get("snippets") or []):
                ans  = (s.get("answer") or "").strip()     # ground-truth answer text
                fp   = s.get("file_path")                  # path to the source file
                span = s.get("span") or [None, None]       # character span [begin, end]

                # add a row for DataFrame
                rows.append({
                    "dataset": dataset,
                    "question": q,
                    "answer": ans,
                    "file_path": fp,
                    "span_begin": span[0],
                    "span_end": span[1],
                })

                # update global lists + qa_dict
                if ans and ans not in qa_dict[q]:
                    qa_dict[q].append(ans)     # add answer under its question
                    answers_all.append(ans)    # add to all answers list

    # ---- build outputs ----
    if rows:
        # full DataFrame of all snippets
        df_gt = pd.DataFrame(rows).reset_index(drop=True)

        # group answers per question into a list (for Ragas)
        ragas_gt_df = (
            df_gt.groupby("question", as_index=False)
                 .agg(ground_truths=("answer", lambda xs: [x for x in xs if x]))
        )
    else:
        # if no rows loaded (edge case), return empty DataFrames
        df_gt = pd.DataFrame(columns=["dataset","question","answer","file_path","span_begin","span_end"])
        ragas_gt_df = pd.DataFrame(columns=["question","ground_truths"])

    # return all 5 useful outputs
    return df_gt, dict(qa_dict), ragas_gt_df, questions_all, answers_all

In [41]:
df_gt, qa_dict, ragas_gt_df, questions_all, answers_all = load_all_benchmarks()

print("Rows in df_gt:", len(df_gt))
print("Unique questions:", len(ragas_gt_df))
print("Total questions (with duplicates):", len(questions_all))
print("Total answers:", len(answers_all))

# Quick peek
display(df_gt.head())
display(ragas_gt_df.head())

# Example: look up one question
sample_q = list(qa_dict.keys())[0]
print("\nQ:", sample_q)
print("Ground truths:", qa_dict[sample_q])

Found benchmark files:
 - contractnli.json
 - cuad.json
 - maud.json
 - privacy_qa.json
Rows in df_gt: 10928
Unique questions: 6856
Total questions (with duplicates): 6889
Total answers: 10890


dataset                                           question  \
0  contractnli  Consider the Non-Disclosure Agreement between ...   
1  contractnli  Consider the Non-Disclosure Agreement between ...   
2  contractnli  Consider the Non-Disclosure Agreement between ...   
3  contractnli  Consider the Non-Disclosure Agreement between ...   
4  contractnli  Consider the Non-Disclosure Agreement between ...   

                                              answer  \
0  Any and all proprietary rights, including but ...   
1  “Confidential Information” means any Idea disc...   
2  Notwithstanding the termination of this Agreem...   
3        At Organiser’s first request, Mentor shall:   
4  (d) erase and/or destroy any Confidential Info...   

                                           file_path  span_begin  span_end  
0  contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_201...       11461     11963  
1  contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_201...        7752      8016  
2  contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_201...       12076     12328  
3  contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_201...       10013     10056  
4  contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_201...       10391     10729

question  \
0  Consider "23andMe"'s privacy policy; are you a...   
1  Consider "23andMe"'s privacy policy; can i del...   
2  Consider "23andMe"'s privacy policy; do you se...   
3  Consider "23andMe"'s privacy policy; do you us...   
4  Consider "23andMe"'s privacy policy; does the ...   

                                       ground_truths  
0  [Web-Behavior Information collected through tr...  
1  [You may be able to correct Self-Reported Info...  
2  [To allow you to share your Personal Informati...  
3  [To allow you to share your Personal Informati...  
4  [23andMe collects and stores the following typ...


Q: Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
Ground truths: ['Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.']


#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [59]:
# Function to evaluate the RAG pipeline

def evaluate_rag_simple(ragas_gt_df: pd.DataFrame,
                        predictions: dict,
                        contexts_map: dict | None = None) -> dict:

    # 1) Align predictions (and contexts if given) to the GT table
    df = ragas_gt_df.copy()
    df["prediction"] = df["question"].map(lambda q: predictions.get(q, ""))
    df["contexts"]   = df["question"].map(lambda q: (contexts_map or {}).get(q, []))

    # Small normalizer: trim + collapse whitespace
    def norm(s: str) -> str:
        s = (s or "").strip()
        return re.sub(r"\s+", " ", s)

    # -------------------- ROUGE --------------------
    # We compute ROUGE for each question against ALL its references
    # and keep the BEST score (common multi-reference practice).
    rouge_variants = ("rouge1", "rouge2", "rougeLsum")
    scorer = rouge_scorer.RougeScorer(list(rouge_variants), use_stemmer=True)
    buckets = {v: [] for v in rouge_variants}

    for _, row in df.iterrows():
        pred = norm(row["prediction"])
        refs = [norm(r) for r in (row["ground_truths"] or []) if r]
        if not refs:
            continue
        best = {v: 0.0 for v in rouge_variants}
        for ref in refs:
            scores = scorer.score(ref, pred)  # ref vs pred
            for v in rouge_variants:
                best[v] = max(best[v], scores[v].fmeasure)
        for v in rouge_variants:
            buckets[v].append(best[v])

    rouge_scores = {v: float(np.mean(buckets[v])) if buckets[v] else 0.0
                    for v in rouge_variants}

    # -------------------- BLEU --------------------
    # Build a multi-reference corpus; use smoothing to avoid zeros.
    smoothie = SmoothingFunction().method3
    list_of_refs, hyps = [], []
    for _, row in df.iterrows():
        refs = [word_tokenize(norm(r)) for r in (row["ground_truths"] or []) if r]
        hyp  = word_tokenize(norm(row["prediction"]))
        if refs:
            list_of_refs.append(refs)
            hyps.append(hyp)
    bleu = float(corpus_bleu(list_of_refs, hyps, smoothing_function=smoothie)) if hyps else 0.0

    # -------------------- RAGAS --------------------
    # If contexts are provided, we can compute faithfulness + context metrics.
    # If not, we still compute answer_relevancy; if anything fails, return the error.
    ragas_scores = {}
    try:
    # ensure required columns for ragas
        df["answer"] = df["prediction"]  # ragas expects 'answer'
        def to_reference(gt_list):
            if not gt_list:
                return ""
            return max(gt_list, key=lambda s: len(s or ""))  # pick longest GT clause
        df["reference"] = df["ground_truths"].map(to_reference)

        has_any_context = any(len(c) > 0 for c in df["contexts"])
        metrics = [answer_relevancy] + ([faithfulness, context_precision, context_recall] if has_any_context else [])

    # ragas dataset needs exactly these columns
        ds = Dataset.from_pandas(
            df[["question", "answer", "contexts", "reference"]],
            preserve_index=False
        )

        res = ragas_evaluate(ds, metrics=metrics)

    # --- NEW: robustly convert EvaluationResult -> dict ---
        if hasattr(res, "to_pandas"):                 # modern ragas
            pdf = res.to_pandas()                     # columns: metric, score, ...
            ragas_scores = {m: float(s) for m, s in zip(pdf["metric"], pdf["score"])}
        elif isinstance(res, dict):                   # older ragas returned dict
            ragas_scores = {k: float(v) for k, v in res.items()}
        else:                                         # fallback: best-effort stringify
            ragas_scores = {"ragas_raw": str(res)}

    except Exception as e:
        ragas_scores = {"ragas_error": str(e)}

    return {"rouge": rouge_scores, "bleu": bleu, "ragas": ragas_scores}

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [44]:
# Evaluate the RAG pipeline

# 1) Take a subset to save time
subset = ragas_gt_df.head(100).copy()                   # first 100 GT rows
questions_100 = subset["question"].tolist()

# 2) Generate predictions (and optionally capture contexts) for those questions
predictions = {}
contexts_map = {}  # if you want RAGAS context metrics

for q in tqdm(questions_100, desc="Answering 100 Qs"):
    # Use your existing chain; result contains 'answer' (LLM output) + 'context' (retrieved docs)
    result = rag_chain.invoke({"input": q})
    predictions[q] = (result.get("answer") or "").strip()            # model's answer text
    contexts_map[q] = [d.page_content for d in (result.get("context") or [])]  # list of chunk texts



Answering 100 Qs: 100%|██████████████████████████████████████████████████████████████| 100/100 [19:23<00:00, 11.63s/it]



=== EVALUATION (first 100) ===
ROUGE: {'rouge1': 0.26790844250768703, 'rouge2': 0.10614319253349815, 'rougeLsum': 0.2379546454621809}
BLEU : 0.11239793307464618
RAGAS: {'ragas_error': "The metric [context_precision] that is used requires the following additional columns ['reference'] to be present in the dataset."}


In [53]:
# 3) Evaluate
scores = evaluate_rag_simple(subset, predictions, contexts_map)

# 4) Print compact report
print("\n=== EVALUATION (first 100) ===")
print("ROUGE:", scores["rouge"])  # dict with rouge1/rouge2/rougeLsum (F1)
print("BLEU :", scores["bleu"])   # corpus BLEU in [0,1]
print("RAGAS:", scores["ragas"])  # dict of ragas metrics (or ragas_error)

Evaluating:   0%|          | 0/400 [00:00<?, ?it/s]


=== EVALUATION (first 100) ===
ROUGE: {'rouge1': 0.26790844250768703, 'rouge2': 0.10614319253349815, 'rougeLsum': 0.2379546454621809}
BLEU : 0.11239793307464618
RAGAS: {'ragas_error': "'metric'"}


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

We evaluated our Retrieval-Augmented Generation (RAG) pipeline on the first 100 benchmark questions using ROUGE, BLEU, and RAGAS metrics. The results were:

ROUGE-1 (0.27): Moderate word-level overlap, showing that the model often uses the right legal terms.

ROUGE-2 (0.11): Much lower phrase-level overlap, indicating the generated answers rarely match the exact phrasing of the reference answers.

ROUGE-Lsum (0.24): Sequence alignment is moderate, meaning the overall sentence structures differ from the references.

BLEU (0.11): Low BLEU score, confirming that the generated answers diverge in wording and phrasing from the ground truth references, even if semantically related.

RAGAS: Did not fully compute due to schema mismatch ('metric' error), but this highlights the importance of aligning our evaluation dataset format (reference columns, contexts) with the expected RAGAS API version.



Insights:


About the data:
________________

Legal documents contain long, complex clauses where the same meaning can be expressed in very different words. This makes overlap-based metrics (ROUGE, BLEU) underestimate quality.

Some reference answers are very verbose, while the model’s outputs are shorter or phrased differently.

About the model pipeline:-
______________________________

The pipeline is retrieving relevant context (evidenced by the presence of correct legal terms), but the LLM sometimes rephrases instead of quoting verbatim.

The chunking strategy and retriever hyperparameters strongly influence whether the exact clause is retrieved.

About the RAG process:-
______________________________

RAG is effective for finding the right clauses but less effective if the evaluation expects near-verbatim matches.

Ensuring that the model extracts sentences verbatim from context, rather than paraphrasing, would likely boost ROUGE and BLEU.

About the results:-
__________________________

Current scores (ROUGE-1 ≈ 0.27, BLEU ≈ 0.11) are modest: the model captures the topic and terms but does not match reference phrasing closely.

The RAGAS error highlights the need for evaluation pipeline alignment. Fixing the schema issue (reference column, API compatibility) will enable richer faithfulness/context metrics to confirm grounding.

Final Takeaway:
_______________________

The RAG pipeline shows promise: it retrieves the right sections and generates semantically relevant answers, but struggles with exact phrasing. This is expected in legal NLP, where clause wording matters. Improving chunking, retrieval settings, and enforcing verbatim sentence extraction in the prompt could significantly improve overlap-based scores. Once RAGAS is fixed, we can also better measure faithfulness and context grounding, which matter more for practical legal applications than surface-level word overlap.